In [26]:
    """
        1 - creat object of orb 
        2 - load images
        3 - create matcher
        4 - find matched points
        5 - finf good matcher
        6 - draw matched
        7 - find homograhy to claculate matrix to make warp prospective
        8- get ROI 
        9- extract text using by teseract
        
Big Thanks to murtaza Hassan

    """
import cv2
import numpy as np
import  matplotlib.pyplot as plt
import os
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'
path = 'C:/Users/Geka/Desktop/haarcascades/form2/'
config = r'--oem 3 --psm 6 '
per =25
myData=[]
pixel_thresh = 49
mypic_list = os.listdir(path)
print(mypic_list)
roi = [[(280, 156), (548, 178), 'text', 'name'], 
 [(279, 193), (548, 213), 'text', 'surename'],
 [(279, 229), (547, 251), 'text', 'midname'], 
 [(280, 268), (548, 289), 'text', 'nationality'],
 [(281, 305), (546, 327), 'text', 'cardnumber'], 
 [(280, 344), (547, 371), 'text', 'passport'], 
 [(280, 387), (545, 416), 'text', 'date'], 
 [(360, 432), (403, 444), 'box', 'male'], 
 [(503, 431), (543, 445), 'box', 'female']]
image =cv2.imread('C:/Users/Geka/Desktop/haarcascades/General-Nursing-Registration-Form.jpg')
h,w,_ = image.shape
grayImage = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

orb = cv2.ORB_create(1000)
kp1,dec1 = orb.detectAndCompute(image,None)
for key,value in enumerate(mypic_list):
    img = cv2.imread(path+value)
    kp2,dec2 = orb.detectAndCompute(img,None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)
    matches = bf.match(dec2,dec1)
    matches =sorted(matches,key=lambda x :x.distance)
    good = matches[:int(len(matches)*(per/100))]
    # imageMatch = cv2.drawMatches(img,kp1,image,kp2,good[:240],None,flage= 2)
    imagesMatches= cv2.drawMatches(img,kp2,image,kp2,good[:240],None,flags=2)
    srcPoints  = np.float32([kp2[m.queryIdx].pt for m in good]).reshape(-1,1,2)
    destPoints = np.float32([kp1[m.trainIdx].pt for m in good]).reshape(-1,1,2)
    M ,_ = cv2.findHomography(srcPoints,destPoints,cv2.RANSAC,5)
    imageScan = cv2.warpPerspective(img,M,(w,h))
    imshow =imageScan.copy()
    imgmask = np.zeros_like(imageScan)
    print(f'##################Extracting Data from form {key+1} #################')
    
    for x,r in enumerate(roi):# [(279, 193), (548, 213), 'text', 'surename'],
        cv2.rectangle(imgmask , (r[0][0],r[0][1]),(r[1][0],r[1][0]),(0,255,0),3)
        imshow = cv2.addWeighted(imshow,.95,imgmask,0.1,0)
        imgCrop = imageScan[r[0][1]:r[1][1],r[0][0]:r[1][0]]#y1:y2 ,x1:x2
        if r[2]=='text':
            myData.append(pytesseract.image_to_string(imgCrop,config='config'))
            print(f'{r[3]}:{pytesseract.image_to_string(imgCrop)}')
        if r[2]=='box':
            gray= cv2.cvtColor(imgCrop,cv2.COLOR_BGR2GRAY)
            thres = cv2.threshold(gray,170,255,cv2.THRESH_BINARY_INV)[1]
            pixel_value = cv2.countNonZero(thres)
            if pixel_value > pixel_thresh : pixel_value=1
            else :   pixel_value=0 
            myData.append(pixel_value)
            print(f'{r[3]}:{pixel_value}')  

       
    # cv2.imshow(str(key),imagesMatches) 
cv2.waitKey(0)
cv2.destroyAllWindows()


['form_1.jpg', 'form_2.jpg']
##################Extracting Data from form 1 #################
name:Ahmed


surename:Geka

midname:WohamedGaber

nationality:Egyptian

cardnumber:123456789

passport:No Paaport agaber@gmail.com

date:07/08/2023

male:1
female:0
##################Extracting Data from form 2 #################
name:rana


surename:roro

midname:alimohamed

nationality:France

cardnumber:987654321

passport:888888-r@yahoo.com

date:15/3/2024

male:0
female:1


In [ ]:
import cv2
import random
circles  = []
point1   = []
point2   = []
mypoints = []
mycolor  = []
counter  = 0
counter2 = 0
scale    =1

def mousePoints(event,x,y,flags,params):
    global point1,point2,counter,counter2 ,mycolor,circles
    if event == cv2.EVENT_LBUTTONDOWN :
        if counter == 0 :
            point1 = int(x//scale),int(y//scale)
            counter +=1
            mycolor = (random.randint(0,2)*200,random.randint(0,2)*200,random.randint(0,2)*200)
        elif counter == 1 :
            point2 = int(x//scale),int(y//scale)
            Type = input('Enter Type')
            name = input('Enter Name')
            mypoints.append([point1,point2,Type,name])
            counter=0
            counter2 +=1

Image =cv2.imread('C:/Users/Geka/Desktop/haarcascades/General-Nursing-Registration-Form.jpg')
Image =cv2.resize(Image,(0,0),None,scale,scale)
while True :
    for x,y,color in circles:
        cv2.circle(Image,(x,y),color,3,cv2.FILLED)
    cv2.imshow('Image',Image)
    cv2.setMouseCallback('Image',mousePoints)
    if cv2.waitKey(0) & 0xFF == ord('q'):
        print(mypoints)
        break
        
cv2.destroyAllWindows()        